## HEIR's Framework Demo

In this notebook, we'll explore HEIR's high-level framework and model of FHE compilation, starting from a cleartext application down to a boolean FHE library backend.

In [1]:
%load_ext heir_play

Loading heir-opt nightly binary


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 61.7M  100 61.7M    0     0  29.7M      0  0:00:02  0:00:02 --:--:-- 50.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Loading heir-translate nightly binary


  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12.7M  100 12.7M    0     0  15.2M      0 --:--:-- --:--:-- --:--:-- 15.2M


### Cleartext Application

For this demo, we'll use a simple toy example that will demonstrate each high-level transformation in the pipeline. The following code models a CMUX on two integers. 

Running `%%heir_opt` with no arguments will validate the IR:

In [2]:
%%heir_opt

module {
  func.func @cmux(%arg0: i16, %arg1: i16, %arg2: i1) -> i16 {
    %0 = scf.if %arg2 -> (i16) {
      scf.yield %arg0 : i16
    } else {
      scf.yield %arg1 : i16
    }
    return %0 : i16
  }
}

Running heir-opt...
module {
  func.func @cmux(%arg0: i16, %arg1: i16, %arg2: i1) -> i16 {
    %0 = scf.if %arg2 -> (i16) {
      scf.yield %arg0 : i16
    } else {
      scf.yield %arg1 : i16
    }
    return %0 : i16
  }
}

None


### Abstracted Private Computation

The first step is to mark which inputs in the IR should be treated as private, or `secret` data.
You can select the private argument(s) by adding the `{secret.secret}` annotation onto the function arguments.

HEIR also has the `--secretize` helper pass, which marks all function inputs with the secret annotation. You can use the pass flag `function=$func` to select the function to secretize.

In [ ]:
%%heir_opt --secretize

module {
  func.func @cmux(%arg0: i16, %arg1: i16, %arg2: i1) -> i16 {
    %0 = scf.if %arg2 -> (i16) {
      scf.yield %arg0 : i16
    } else {
      scf.yield %arg1 : i16
    }
    return %0 : i16
  }
}

Running heir-opt...
module {
  func.func @cmux(%arg0: i16 {secret.secret}, %arg1: i16 {secret.secret}, %arg2: i1 {secret.secret}) -> i16 {
    %0 = scf.if %arg2 -> (i16) {
      scf.yield %arg0 : i16
    } else {
      scf.yield %arg1 : i16
    }
    return %0 : i16
  }
}

None


#### Modeling Cleartext Computation on Private Data 

Once the computation is annotated with secrets, the `--wrap-generic` pass will convert plaintext integer types into data types from the [secret dialect](https://heir.dev/docs/dialects/secret/).

A secret data type wraps an underlying MLIR type, like `!secret.secret<i16>`. However, the cleartext operations used in the original IR don't allow these new secret data types, making it difficult to utilize upstream optimizations and canonicalization on dialects like `arith`. So we wrap cleartext computation in `secret.generic` operations that lift the inner cleartext arguments to their corresponding secret data arguments. This [page](https://heir.dev/docs/design/secret/) discusses more information about secret dialect's design.

In [4]:
%%heir_opt --wrap-generic
module {
  func.func @cmux(%arg0: i16 {secret.secret}, %arg1: i16 {secret.secret}, %arg2: i1 {secret.secret}) -> i16 {
    %0 = scf.if %arg2 -> (i16) {
      scf.yield %arg0 : i16
    } else {
      scf.yield %arg1 : i16
    }
    return %0 : i16
  }
} 

Running heir-opt...
module {
  func.func @cmux(%arg0: !secret.secret<i16>, %arg1: !secret.secret<i16>, %arg2: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = secret.generic ins(%arg0, %arg1, %arg2 : !secret.secret<i16>, !secret.secret<i16>, !secret.secret<i1>) {
    ^bb0(%arg3: i16, %arg4: i16, %arg5: i1):
      %1 = scf.if %arg5 -> (i16) {
        scf.yield %arg3 : i16
      } else {
        scf.yield %arg4 : i16
      }
      secret.yield %1 : i16
    } -> !secret.secret<i16>
    return %0 : !secret.secret<i16>
  }
}

None


Clearly, the `secret.generic` operation did not do anything except wrap the original computation block and map secret data arguments to inner cleartext ones.

#### Data-Oblivious Transforms

Did you notice that we have a secret conditional in our IR? FHE programs can't support data-dependent control flow! Next, we'll transform the program to an equivalent, "data-oblivious" program.

The pass pipeline `--convert-to-data-oblivious` performs a number of transformations:
* Data-dependent `if` statement to `arith.select` statements.
* Data-dependent loop statements to static input-independent loops and early-exits.
* Data-dependent memory access to static read-write operations over the full structure.

You can read more about the transformations in detail [here](https://heir.dev/docs/design/do_transformation/).

In [5]:
%%heir_opt --convert-to-data-oblivious
module {
  func.func @cmux(%arg0: !secret.secret<i16>, %arg1: !secret.secret<i16>, %arg2: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = secret.generic ins(%arg0, %arg1, %arg2 : !secret.secret<i16>, !secret.secret<i16>, !secret.secret<i1>) {
    ^bb0(%arg3: i16, %arg4: i16, %arg5: i1):
      %1 = scf.if %arg5 -> (i16) {
        scf.yield %arg3 : i16
      } else {
        scf.yield %arg4 : i16
      }
      secret.yield %1 : i16
    } -> !secret.secret<i16>
    return %0 : !secret.secret<i16>
  }
}

Running heir-opt...
module {
  func.func @cmux(%arg0: !secret.secret<i16>, %arg1: !secret.secret<i16>, %arg2: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = secret.generic ins(%arg0, %arg1, %arg2 : !secret.secret<i16>, !secret.secret<i16>, !secret.secret<i1>) {
    ^bb0(%arg3: i16, %arg4: i16, %arg5: i1):
      %1 = arith.select %arg5, %arg3, %arg4 : i16
      secret.yield %1 : i16
    } -> !secret.secret<i16>
    return %0 : !secret.secret<i16>
  }
}

None


### Aside: Plaintext Execution

At any point during the pipeline, you may also want to test the plaintext application for functional correctness or for measuring the added overhead from the transformations we've applied.

HEIR has a `--secret-forget-secrets` pass that unwraps all secret data types and removes the `secret.generic` blocks so that the computation runs using only plaintext data types and operations.

In [29]:
%%heir_opt --secret-forget-secrets
module {
  func.func @cmux(%arg0: !secret.secret<i16>, %arg1: !secret.secret<i16>, %arg2: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = secret.generic ins(%arg0, %arg1, %arg2 : !secret.secret<i16>, !secret.secret<i16>, !secret.secret<i1>) {
    ^bb0(%arg3: i16, %arg4: i16, %arg5: i1):
      %1 = arith.select %arg5, %arg3, %arg4 : i16
      secret.yield %1 : i16
    } -> !secret.secret<i16>
    return %0 : !secret.secret<i16>
  }
}

Running heir-opt...
module {
  func.func @cmux(%arg0: i16, %arg1: i16, %arg2: i1) -> i16 {
    %0 = arith.select %arg2, %arg0, %arg1 : i16
    return %0 : i16
  }
}

None


You can compile the plaintext program by:
* Lowering to LLVM IR using upstream passes
* Covnvert the LLVM IR to LLVM bytecode using `mlir-translate --mlir-to-llvmir`
* Use LLVM static compiler `llc` to generate assembly code

HEIR includes bazel [macros](https://github.com/google/heir/blob/90796977be9b7da90f5dd8bd8a9fbfc75039cc33/tests/Dialect/Polynomial/Conversions/heir_polynomial_to_llvm/runner/BUILD#L20) that packages this entire process (including the HEIR transforms) into a single build invocation. See [this](https://github.com/google/heir/blob/90796977be9b7da90f5dd8bd8a9fbfc75039cc33/tests/Dialect/Polynomial/Conversions/heir_polynomial_to_llvm/runner/BUILD#L20) example.

#### Booleanization

Since we're targeting a boolean scheme, the last step we'll need to create a valid plaintext program that's ready for ciphertext conversion is to convert arithmetic operations on high bit width data types into boolean operations like gates and lookup tables.

HEIR uses a conversion pass `--yosys-optimizer` that runs [Yosys](https://yosyshq.net/yosys/) and [abc](https://github.com/berkeley-abc/abc), a logic synthesis and optimizer tool, to convert from arithmetic programs to optimized boolean circuits.

In [8]:
%%heir_opt --yosys-optimizer=mode=Boolean

module {
  func.func @cmux(%arg0: !secret.secret<i16>, %arg1: !secret.secret<i16>, %arg2: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = secret.generic ins(%arg0, %arg1, %arg2 : !secret.secret<i16>, !secret.secret<i16>, !secret.secret<i1>) {
    ^bb0(%arg3: i16, %arg4: i16, %arg5: i1):
      %1 = arith.select %arg5, %arg3, %arg4 : i16
      secret.yield %1 : i16
    } -> !secret.secret<i16>
    return %0 : !secret.secret<i16>
  }
}

Running heir-opt...
module {
  func.func @cmux(%arg0: !secret.secret<i16>, %arg1: !secret.secret<i16>, %arg2: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = secret.cast %arg0 : !secret.secret<i16> to !secret.secret<memref<16xi1>>
    %1 = secret.cast %arg1 : !secret.secret<i16> to !secret.secret<memref<16xi1>>
    %2 = secret.cast %arg2 : !secret.secret<i1> to !secret.secret<i1>
    %3 = secret.generic ins(%0, %1, %2 : !secret.secret<memref<16xi1>>, !secret.secret<memref<16xi1>>, !secret.secret<i1>) {
    ^bb0(%arg3: memref<16xi1>, %arg4: memref<16xi1>, %arg5: i1):
      %c1 = arith.constant 1 : index
      %5 = memref.load %arg3[%c1] : memref<16xi1>
      %6 = comb.nand %arg5, %5 : i1
      %7 = comb.inv %arg5 : i1
      %c4 = arith.constant 4 : index
      %8 = memref.load %arg4[%c4] : memref<16xi1>
      %9 = comb.nand %7, %8 : i1
      %10 = memref.load %arg3[%c4] : memref<16xi1>
      %11 = comb.nand %arg5, %10 : i1
      %12 = comb.nand %11, %9 : i1
      %c5 = arith.const

#### Ciphertext program conversion

Great! Now we have a program that uses operations from the `comb` dialect and operate on boolean data types. At this stage, we're ready to convert to a ciphertext program in the CGGI dialect.

The CGGI dialet in HEIR models operations of the CGGI scheme and uses LWE plaintext and ciphertext data types that all boolean and arithmetic scheme dialects use.

In [9]:
%%heir_opt --secret-distribute-generic --secret-to-cggi --cse
module {
  func.func @cmux(%arg0: !secret.secret<i16>, %arg1: !secret.secret<i16>, %arg2: !secret.secret<i1>) -> !secret.secret<i16> {
    %0 = secret.cast %arg0 : !secret.secret<i16> to !secret.secret<memref<16xi1>>
    %1 = secret.cast %arg1 : !secret.secret<i16> to !secret.secret<memref<16xi1>>
    %2 = secret.cast %arg2 : !secret.secret<i1> to !secret.secret<i1>
    %3 = secret.generic ins(%0, %1, %2 : !secret.secret<memref<16xi1>>, !secret.secret<memref<16xi1>>, !secret.secret<i1>) {
    ^bb0(%arg3: memref<16xi1>, %arg4: memref<16xi1>, %arg5: i1):
      %c1 = arith.constant 1 : index
      %5 = memref.load %arg3[%c1] : memref<16xi1>
      %6 = comb.nand %arg5, %5 : i1
      %7 = comb.inv %arg5 : i1
      %c4 = arith.constant 4 : index
      %8 = memref.load %arg4[%c4] : memref<16xi1>
      %9 = comb.nand %7, %8 : i1
      %10 = memref.load %arg3[%c4] : memref<16xi1>
      %11 = comb.nand %arg5, %10 : i1
      %12 = comb.nand %11, %9 : i1
      %c5 = arith.constant 5 : index
      %13 = memref.load %arg3[%c5] : memref<16xi1>
      %14 = comb.nand %arg5, %13 : i1
      %15 = memref.load %arg4[%c5] : memref<16xi1>
      %16 = comb.nand %7, %15 : i1
      %17 = comb.nand %14, %16 : i1
      %c6 = arith.constant 6 : index
      %18 = memref.load %arg3[%c6] : memref<16xi1>
      %19 = comb.nand %arg5, %18 : i1
      %20 = memref.load %arg4[%c6] : memref<16xi1>
      %21 = comb.nand %7, %20 : i1
      %22 = comb.nand %19, %21 : i1
      %c7 = arith.constant 7 : index
      %23 = memref.load %arg3[%c7] : memref<16xi1>
      %24 = comb.nand %arg5, %23 : i1
      %25 = memref.load %arg4[%c7] : memref<16xi1>
      %26 = comb.nand %7, %25 : i1
      %27 = memref.load %arg4[%c1] : memref<16xi1>
      %28 = comb.nand %7, %27 : i1
      %29 = comb.nand %24, %26 : i1
      %c8 = arith.constant 8 : index
      %30 = memref.load %arg3[%c8] : memref<16xi1>
      %31 = comb.nand %arg5, %30 : i1
      %32 = memref.load %arg4[%c8] : memref<16xi1>
      %33 = comb.nand %7, %32 : i1
      %34 = comb.nand %31, %33 : i1
      %c9 = arith.constant 9 : index
      %35 = memref.load %arg3[%c9] : memref<16xi1>
      %36 = comb.nand %arg5, %35 : i1
      %37 = memref.load %arg4[%c9] : memref<16xi1>
      %38 = comb.nand %7, %37 : i1
      %39 = comb.nand %36, %38 : i1
      %c10 = arith.constant 10 : index
      %40 = memref.load %arg3[%c10] : memref<16xi1>
      %41 = comb.nand %arg5, %40 : i1
      %42 = memref.load %arg4[%c10] : memref<16xi1>
      %43 = comb.nand %7, %42 : i1
      %44 = comb.nand %41, %43 : i1
      %45 = comb.nand %6, %28 : i1
      %c11 = arith.constant 11 : index
      %46 = memref.load %arg3[%c11] : memref<16xi1>
      %47 = comb.nand %arg5, %46 : i1
      %48 = memref.load %arg4[%c11] : memref<16xi1>
      %49 = comb.nand %7, %48 : i1
      %50 = comb.nand %47, %49 : i1
      %c12 = arith.constant 12 : index
      %51 = memref.load %arg3[%c12] : memref<16xi1>
      %52 = comb.nand %arg5, %51 : i1
      %53 = memref.load %arg4[%c12] : memref<16xi1>
      %54 = comb.nand %7, %53 : i1
      %55 = comb.nand %52, %54 : i1
      %c13 = arith.constant 13 : index
      %56 = memref.load %arg3[%c13] : memref<16xi1>
      %57 = comb.nand %arg5, %56 : i1
      %58 = memref.load %arg4[%c13] : memref<16xi1>
      %59 = comb.nand %7, %58 : i1
      %60 = comb.nand %57, %59 : i1
      %c14 = arith.constant 14 : index
      %61 = memref.load %arg3[%c14] : memref<16xi1>
      %62 = comb.nand %arg5, %61 : i1
      %c2 = arith.constant 2 : index
      %63 = memref.load %arg3[%c2] : memref<16xi1>
      %64 = comb.nand %arg5, %63 : i1
      %65 = memref.load %arg4[%c14] : memref<16xi1>
      %66 = comb.nand %7, %65 : i1
      %67 = comb.nand %62, %66 : i1
      %c15 = arith.constant 15 : index
      %68 = memref.load %arg3[%c15] : memref<16xi1>
      %69 = comb.nand %arg5, %68 : i1
      %70 = memref.load %arg4[%c15] : memref<16xi1>
      %71 = comb.nand %7, %70 : i1
      %72 = comb.nand %69, %71 : i1
      %c0 = arith.constant 0 : index
      %73 = memref.load %arg3[%c0] : memref<16xi1>
      %74 = comb.nand %73, %arg5 : i1
      %75 = memref.load %arg4[%c0] : memref<16xi1>
      %76 = comb.nand %75, %7 : i1
      %77 = comb.nand %74, %76 : i1
      %78 = memref.load %arg4[%c2] : memref<16xi1>
      %79 = comb.nand %7, %78 : i1
      %80 = comb.nand %64, %79 : i1
      %c3 = arith.constant 3 : index
      %81 = memref.load %arg3[%c3] : memref<16xi1>
      %82 = comb.nand %arg5, %81 : i1
      %83 = memref.load %arg4[%c3] : memref<16xi1>
      %84 = comb.nand %7, %83 : i1
      %85 = comb.nand %82, %84 : i1
      %alloc = memref.alloc() : memref<16xi1>
      memref.store %77, %alloc[%c0] : memref<16xi1>
      memref.store %45, %alloc[%c1] : memref<16xi1>
      memref.store %80, %alloc[%c2] : memref<16xi1>
      memref.store %85, %alloc[%c3] : memref<16xi1>
      memref.store %12, %alloc[%c4] : memref<16xi1>
      memref.store %17, %alloc[%c5] : memref<16xi1>
      memref.store %22, %alloc[%c6] : memref<16xi1>
      memref.store %29, %alloc[%c7] : memref<16xi1>
      memref.store %34, %alloc[%c8] : memref<16xi1>
      memref.store %39, %alloc[%c9] : memref<16xi1>
      memref.store %44, %alloc[%c10] : memref<16xi1>
      memref.store %50, %alloc[%c11] : memref<16xi1>
      memref.store %55, %alloc[%c12] : memref<16xi1>
      memref.store %60, %alloc[%c13] : memref<16xi1>
      memref.store %67, %alloc[%c14] : memref<16xi1>
      memref.store %72, %alloc[%c15] : memref<16xi1>
      secret.yield %alloc : memref<16xi1>
    } -> !secret.secret<memref<16xi1>>
    %4 = secret.cast %3 : !secret.secret<memref<16xi1>> to !secret.secret<i16>
    return %4 : !secret.secret<i16>
  }
}


Running heir-opt...
module {
  func.func @cmux(%arg0: memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>, %arg1: memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>, %arg2: !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>) -> memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>> {
    %c3 = arith.constant 3 : index
    %c0 = arith.constant 0 : index
    %c15 = arith.constant 15 : index
    %c2 = arith.constant 2 : index
    %c14 = arith.constant 14 : index
    %c13 = arith.constant 13 : index
    %c12 = arith.constant 12 : index
    %c11 = arith.constant 11 : index
    %c10 = arith.constant 10 : index
    %c9 = arith.constant 9 : index
    %c8 = arith.constant 8 : index
    %c7 = arith.constant 7 : index
    %c6 = arith.constant 6 : index
    %c5 = arith.constant 5 : index
    %c4 = arith.const

#### Backend Library Conversion

Now that we have program expressed with CGGI scheme operations and data, we can convert to an implementation of the CGGI scheme. For example, we can utilize dialects the mirror CGGI libraries like tfhe-rs or OpenFHE, or we can convert to polynomial operations that can be executed on CPU or other hardware. We'll use `heir-opt` to convert the CGGI scheme to the `tfhe_rust` dialect, which mirrors Zama's `tfhe-rs`

In [12]:
%%heir_opt --cggi-to-tfhe-rust-bool --canonicalize --cse

module {
  func.func @cmux(%arg0: memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>, %arg1: memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>, %arg2: !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>) -> memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>> {
    %c3 = arith.constant 3 : index
    %c0 = arith.constant 0 : index
    %c15 = arith.constant 15 : index
    %c2 = arith.constant 2 : index
    %c14 = arith.constant 14 : index
    %c13 = arith.constant 13 : index
    %c12 = arith.constant 12 : index
    %c11 = arith.constant 11 : index
    %c10 = arith.constant 10 : index
    %c9 = arith.constant 9 : index
    %c8 = arith.constant 8 : index
    %c7 = arith.constant 7 : index
    %c6 = arith.constant 6 : index
    %c5 = arith.constant 5 : index
    %c4 = arith.constant 4 : index
    %c1 = arith.constant 1 : index
    %0 = memref.load %arg0[%c1] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %1 = cggi.nand %arg2, %0 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %2 = cggi.not %arg2 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %3 = memref.load %arg1[%c4] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %4 = cggi.nand %2, %3 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %5 = memref.load %arg0[%c4] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %6 = cggi.nand %arg2, %5 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %7 = cggi.nand %6, %4 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %8 = memref.load %arg0[%c5] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %9 = cggi.nand %arg2, %8 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %10 = memref.load %arg1[%c5] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %11 = cggi.nand %2, %10 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %12 = cggi.nand %9, %11 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %13 = memref.load %arg0[%c6] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %14 = cggi.nand %arg2, %13 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %15 = memref.load %arg1[%c6] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %16 = cggi.nand %2, %15 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %17 = cggi.nand %14, %16 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %18 = memref.load %arg0[%c7] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %19 = cggi.nand %arg2, %18 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %20 = memref.load %arg1[%c7] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %21 = cggi.nand %2, %20 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %22 = memref.load %arg1[%c1] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %23 = cggi.nand %2, %22 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %24 = cggi.nand %19, %21 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %25 = memref.load %arg0[%c8] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %26 = cggi.nand %arg2, %25 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %27 = memref.load %arg1[%c8] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %28 = cggi.nand %2, %27 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %29 = cggi.nand %26, %28 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %30 = memref.load %arg0[%c9] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %31 = cggi.nand %arg2, %30 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %32 = memref.load %arg1[%c9] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %33 = cggi.nand %2, %32 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %34 = cggi.nand %31, %33 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %35 = memref.load %arg0[%c10] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %36 = cggi.nand %arg2, %35 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %37 = memref.load %arg1[%c10] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %38 = cggi.nand %2, %37 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %39 = cggi.nand %36, %38 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %40 = cggi.nand %1, %23 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %41 = memref.load %arg0[%c11] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %42 = cggi.nand %arg2, %41 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %43 = memref.load %arg1[%c11] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %44 = cggi.nand %2, %43 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %45 = cggi.nand %42, %44 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %46 = memref.load %arg0[%c12] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %47 = cggi.nand %arg2, %46 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %48 = memref.load %arg1[%c12] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %49 = cggi.nand %2, %48 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %50 = cggi.nand %47, %49 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %51 = memref.load %arg0[%c13] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %52 = cggi.nand %arg2, %51 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %53 = memref.load %arg1[%c13] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %54 = cggi.nand %2, %53 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %55 = cggi.nand %52, %54 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %56 = memref.load %arg0[%c14] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %57 = cggi.nand %arg2, %56 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %58 = memref.load %arg0[%c2] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %59 = cggi.nand %arg2, %58 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %60 = memref.load %arg1[%c14] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %61 = cggi.nand %2, %60 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %62 = cggi.nand %57, %61 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %63 = memref.load %arg0[%c15] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %64 = cggi.nand %arg2, %63 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %65 = memref.load %arg1[%c15] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %66 = cggi.nand %2, %65 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %67 = cggi.nand %64, %66 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %68 = memref.load %arg0[%c0] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %69 = cggi.nand %68, %arg2 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %70 = memref.load %arg1[%c0] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %71 = cggi.nand %70, %2 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %72 = cggi.nand %69, %71 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %73 = memref.load %arg1[%c2] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %74 = cggi.nand %2, %73 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %75 = cggi.nand %59, %74 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %76 = memref.load %arg0[%c3] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %77 = cggi.nand %arg2, %76 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %78 = memref.load %arg1[%c3] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    %79 = cggi.nand %2, %78 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %80 = cggi.nand %77, %79 : !lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>
    %alloc = memref.alloc() : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %72, %alloc[%c0] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %40, %alloc[%c1] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %75, %alloc[%c2] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %80, %alloc[%c3] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %7, %alloc[%c4] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %12, %alloc[%c5] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %17, %alloc[%c6] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %24, %alloc[%c7] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %29, %alloc[%c8] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %34, %alloc[%c9] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %39, %alloc[%c10] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %45, %alloc[%c11] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %50, %alloc[%c12] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %55, %alloc[%c13] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %62, %alloc[%c14] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    memref.store %67, %alloc[%c15] : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
    return %alloc : memref<16x!lwe.lwe_ciphertext<encoding = #lwe.unspecified_bit_field_encoding<cleartext_bitwidth = 1>>>
  }
}


Running heir-opt...
module {
  func.func @cmux(%arg0: !tfhe_rust_bool.server_key, %arg1: memref<16x!tfhe_rust_bool.eb>, %arg2: memref<16x!tfhe_rust_bool.eb>, %arg3: !tfhe_rust_bool.eb) -> memref<16x!tfhe_rust_bool.eb> {
    %c3 = arith.constant 3 : index
    %c0 = arith.constant 0 : index
    %c15 = arith.constant 15 : index
    %c2 = arith.constant 2 : index
    %c14 = arith.constant 14 : index
    %c13 = arith.constant 13 : index
    %c12 = arith.constant 12 : index
    %c11 = arith.constant 11 : index
    %c10 = arith.constant 10 : index
    %c9 = arith.constant 9 : index
    %c8 = arith.constant 8 : index
    %c7 = arith.constant 7 : index
    %c6 = arith.constant 6 : index
    %c5 = arith.constant 5 : index
    %c4 = arith.constant 4 : index
    %c1 = arith.constant 1 : index
    %0 = memref.load %arg1[%c1] : memref<16x!tfhe_rust_bool.eb>
    %1 = tfhe_rust_bool.nand %arg0, %arg3, %0 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %

#### Code Generation

Finally, we'll use `heir-translate` to convert the library dialect into rust code that can be compiled and run. 

In [13]:
%%heir_translate --emit-tfhe-rust-bool

module {
  func.func @cmux(%arg0: !tfhe_rust_bool.server_key, %arg1: memref<16x!tfhe_rust_bool.eb>, %arg2: memref<16x!tfhe_rust_bool.eb>, %arg3: !tfhe_rust_bool.eb) -> memref<16x!tfhe_rust_bool.eb> {
    %c3 = arith.constant 3 : index
    %c0 = arith.constant 0 : index
    %c15 = arith.constant 15 : index
    %c2 = arith.constant 2 : index
    %c14 = arith.constant 14 : index
    %c13 = arith.constant 13 : index
    %c12 = arith.constant 12 : index
    %c11 = arith.constant 11 : index
    %c10 = arith.constant 10 : index
    %c9 = arith.constant 9 : index
    %c8 = arith.constant 8 : index
    %c7 = arith.constant 7 : index
    %c6 = arith.constant 6 : index
    %c5 = arith.constant 5 : index
    %c4 = arith.constant 4 : index
    %c1 = arith.constant 1 : index
    %0 = memref.load %arg1[%c1] : memref<16x!tfhe_rust_bool.eb>
    %1 = tfhe_rust_bool.nand %arg0, %arg3, %0 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %2 = tfhe_rust_bool.not %arg0, %arg3 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %3 = memref.load %arg2[%c4] : memref<16x!tfhe_rust_bool.eb>
    %4 = tfhe_rust_bool.nand %arg0, %2, %3 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %5 = memref.load %arg1[%c4] : memref<16x!tfhe_rust_bool.eb>
    %6 = tfhe_rust_bool.nand %arg0, %arg3, %5 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %7 = tfhe_rust_bool.nand %arg0, %6, %4 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %8 = memref.load %arg1[%c5] : memref<16x!tfhe_rust_bool.eb>
    %9 = tfhe_rust_bool.nand %arg0, %arg3, %8 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %10 = memref.load %arg2[%c5] : memref<16x!tfhe_rust_bool.eb>
    %11 = tfhe_rust_bool.nand %arg0, %2, %10 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %12 = tfhe_rust_bool.nand %arg0, %9, %11 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %13 = memref.load %arg1[%c6] : memref<16x!tfhe_rust_bool.eb>
    %14 = tfhe_rust_bool.nand %arg0, %arg3, %13 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %15 = memref.load %arg2[%c6] : memref<16x!tfhe_rust_bool.eb>
    %16 = tfhe_rust_bool.nand %arg0, %2, %15 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %17 = tfhe_rust_bool.nand %arg0, %14, %16 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %18 = memref.load %arg1[%c7] : memref<16x!tfhe_rust_bool.eb>
    %19 = tfhe_rust_bool.nand %arg0, %arg3, %18 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %20 = memref.load %arg2[%c7] : memref<16x!tfhe_rust_bool.eb>
    %21 = tfhe_rust_bool.nand %arg0, %2, %20 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %22 = memref.load %arg2[%c1] : memref<16x!tfhe_rust_bool.eb>
    %23 = tfhe_rust_bool.nand %arg0, %2, %22 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %24 = tfhe_rust_bool.nand %arg0, %19, %21 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %25 = memref.load %arg1[%c8] : memref<16x!tfhe_rust_bool.eb>
    %26 = tfhe_rust_bool.nand %arg0, %arg3, %25 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %27 = memref.load %arg2[%c8] : memref<16x!tfhe_rust_bool.eb>
    %28 = tfhe_rust_bool.nand %arg0, %2, %27 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %29 = tfhe_rust_bool.nand %arg0, %26, %28 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %30 = memref.load %arg1[%c9] : memref<16x!tfhe_rust_bool.eb>
    %31 = tfhe_rust_bool.nand %arg0, %arg3, %30 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %32 = memref.load %arg2[%c9] : memref<16x!tfhe_rust_bool.eb>
    %33 = tfhe_rust_bool.nand %arg0, %2, %32 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %34 = tfhe_rust_bool.nand %arg0, %31, %33 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %35 = memref.load %arg1[%c10] : memref<16x!tfhe_rust_bool.eb>
    %36 = tfhe_rust_bool.nand %arg0, %arg3, %35 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %37 = memref.load %arg2[%c10] : memref<16x!tfhe_rust_bool.eb>
    %38 = tfhe_rust_bool.nand %arg0, %2, %37 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %39 = tfhe_rust_bool.nand %arg0, %36, %38 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %40 = tfhe_rust_bool.nand %arg0, %1, %23 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %41 = memref.load %arg1[%c11] : memref<16x!tfhe_rust_bool.eb>
    %42 = tfhe_rust_bool.nand %arg0, %arg3, %41 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %43 = memref.load %arg2[%c11] : memref<16x!tfhe_rust_bool.eb>
    %44 = tfhe_rust_bool.nand %arg0, %2, %43 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %45 = tfhe_rust_bool.nand %arg0, %42, %44 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %46 = memref.load %arg1[%c12] : memref<16x!tfhe_rust_bool.eb>
    %47 = tfhe_rust_bool.nand %arg0, %arg3, %46 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %48 = memref.load %arg2[%c12] : memref<16x!tfhe_rust_bool.eb>
    %49 = tfhe_rust_bool.nand %arg0, %2, %48 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %50 = tfhe_rust_bool.nand %arg0, %47, %49 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %51 = memref.load %arg1[%c13] : memref<16x!tfhe_rust_bool.eb>
    %52 = tfhe_rust_bool.nand %arg0, %arg3, %51 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %53 = memref.load %arg2[%c13] : memref<16x!tfhe_rust_bool.eb>
    %54 = tfhe_rust_bool.nand %arg0, %2, %53 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %55 = tfhe_rust_bool.nand %arg0, %52, %54 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %56 = memref.load %arg1[%c14] : memref<16x!tfhe_rust_bool.eb>
    %57 = tfhe_rust_bool.nand %arg0, %arg3, %56 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %58 = memref.load %arg1[%c2] : memref<16x!tfhe_rust_bool.eb>
    %59 = tfhe_rust_bool.nand %arg0, %arg3, %58 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %60 = memref.load %arg2[%c14] : memref<16x!tfhe_rust_bool.eb>
    %61 = tfhe_rust_bool.nand %arg0, %2, %60 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %62 = tfhe_rust_bool.nand %arg0, %57, %61 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %63 = memref.load %arg1[%c15] : memref<16x!tfhe_rust_bool.eb>
    %64 = tfhe_rust_bool.nand %arg0, %arg3, %63 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %65 = memref.load %arg2[%c15] : memref<16x!tfhe_rust_bool.eb>
    %66 = tfhe_rust_bool.nand %arg0, %2, %65 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %67 = tfhe_rust_bool.nand %arg0, %64, %66 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %68 = memref.load %arg1[%c0] : memref<16x!tfhe_rust_bool.eb>
    %69 = tfhe_rust_bool.nand %arg0, %68, %arg3 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %70 = memref.load %arg2[%c0] : memref<16x!tfhe_rust_bool.eb>
    %71 = tfhe_rust_bool.nand %arg0, %70, %2 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %72 = tfhe_rust_bool.nand %arg0, %69, %71 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %73 = memref.load %arg2[%c2] : memref<16x!tfhe_rust_bool.eb>
    %74 = tfhe_rust_bool.nand %arg0, %2, %73 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %75 = tfhe_rust_bool.nand %arg0, %59, %74 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %76 = memref.load %arg1[%c3] : memref<16x!tfhe_rust_bool.eb>
    %77 = tfhe_rust_bool.nand %arg0, %arg3, %76 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %78 = memref.load %arg2[%c3] : memref<16x!tfhe_rust_bool.eb>
    %79 = tfhe_rust_bool.nand %arg0, %2, %78 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %80 = tfhe_rust_bool.nand %arg0, %77, %79 : (!tfhe_rust_bool.server_key, !tfhe_rust_bool.eb, !tfhe_rust_bool.eb) -> !tfhe_rust_bool.eb
    %alloc = memref.alloc() : memref<16x!tfhe_rust_bool.eb>
    memref.store %72, %alloc[%c0] : memref<16x!tfhe_rust_bool.eb>
    memref.store %40, %alloc[%c1] : memref<16x!tfhe_rust_bool.eb>
    memref.store %75, %alloc[%c2] : memref<16x!tfhe_rust_bool.eb>
    memref.store %80, %alloc[%c3] : memref<16x!tfhe_rust_bool.eb>
    memref.store %7, %alloc[%c4] : memref<16x!tfhe_rust_bool.eb>
    memref.store %12, %alloc[%c5] : memref<16x!tfhe_rust_bool.eb>
    memref.store %17, %alloc[%c6] : memref<16x!tfhe_rust_bool.eb>
    memref.store %24, %alloc[%c7] : memref<16x!tfhe_rust_bool.eb>
    memref.store %29, %alloc[%c8] : memref<16x!tfhe_rust_bool.eb>
    memref.store %34, %alloc[%c9] : memref<16x!tfhe_rust_bool.eb>
    memref.store %39, %alloc[%c10] : memref<16x!tfhe_rust_bool.eb>
    memref.store %45, %alloc[%c11] : memref<16x!tfhe_rust_bool.eb>
    memref.store %50, %alloc[%c12] : memref<16x!tfhe_rust_bool.eb>
    memref.store %55, %alloc[%c13] : memref<16x!tfhe_rust_bool.eb>
    memref.store %62, %alloc[%c14] : memref<16x!tfhe_rust_bool.eb>
    memref.store %67, %alloc[%c15] : memref<16x!tfhe_rust_bool.eb>
    return %alloc : memref<16x!tfhe_rust_bool.eb>
  }
}

Running heir-translate...

use std::collections::BTreeMap;
use tfhe::boolean::prelude::*;

pub fn cmux(
  v0: &ServerKey,
  v1: &Vec<Ciphertext>,
  v2: &Vec<Ciphertext>,
  v3: &Ciphertext,
) -> Vec<Ciphertext> {
  let v4 = 3;
  let v5 = 0;
  let v6 = 15;
  let v7 = 2;
  let v8 = 14;
  let v9 = 13;
  let v10 = 12;
  let v11 = 11;
  let v12 = 10;
  let v13 = 9;
  let v14 = 8;
  let v15 = 7;
  let v16 = 6;
  let v17 = 5;
  let v18 = 4;
  let v19 = 1;
  let v20 = &v1[v19];
  let v21 = v0.nand(v3, v20);
  let v22 = v0.not(v3);
  let v23 = &v2[v18];
  let v24 = v0.nand(&v22, v23);
  let v25 = &v1[v18];
  let v26 = v0.nand(v3, v25);
  let v27 = v0.nand(&v26, &v24);
  let v28 = &v1[v17];
  let v29 = v0.nand(v3, v28);
  let v30 = &v2[v17];
  let v31 = v0.nand(&v22, v30);
  let v32 = v0.nand(&v29, &v31);
  let v33 = &v1[v16];
  let v34 = v0.nand(v3, v33);
  let v35 = &v2[v16];
  let v36 = v0.nand(&v22, v35);
  let v37 = v0.nand(&v34, &v36);
  let v38 = &v1[v15];
  let v39 = v0.nand(v3, v38);
  l

### Final Notes

We also include a bazel macros that bundle this process for different library backends, and build a library with the required dependencies for each backend. See the `tests/Examples/<BACKEND>` folders for uses.